In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("LogisticRegression").getOrCreate()

23/12/27 10:15:29 WARN Utils: Your hostname, tkroza-IdeaPad-L340-15IRH-Gaming resolves to a loopback address: 127.0.1.1; using 192.168.15.10 instead (on interface wlp7s0)
23/12/27 10:15:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/27 10:15:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
churn = spark.read.csv("/home/tkroza/PycharmProjects/ML_with_pyspark/data/Churn.csv", header=True, inferSchema=True, sep=";")
print(churn.count())
churn.show(5)

10000
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+-

In [5]:
from pyspark.ml.feature import RFormula

Rformula = RFormula(formula="Exited ~ .", featuresCol="independente", labelCol="dependente")
churnrf = Rformula.fit(churn).transform(churn)
churnrf.select("independente", "dependente").show(5, truncate=False)

+--------------------------------------------------------------+----------+
|independente                                                  |dependente|
+--------------------------------------------------------------+----------+
|[619.0,1.0,0.0,0.0,42.0,2.0,0.0,1.0,1.0,1.0,1.0134888E7]      |1.0       |
|[608.0,0.0,0.0,0.0,41.0,1.0,8380786.0,1.0,0.0,1.0,1.1254258E7]|0.0       |
|[502.0,1.0,0.0,0.0,42.0,8.0,1596608.0,3.0,1.0,0.0,1.1393157E7]|1.0       |
|(11,[0,1,4,5,7,10],[699.0,1.0,39.0,1.0,2.0,9382663.0])        |0.0       |
|[850.0,0.0,0.0,0.0,43.0,2.0,1.2551082E7,1.0,1.0,1.0,790841.0] |0.0       |
+--------------------------------------------------------------+----------+
only showing top 5 rows



In [6]:
churnTreino, churnTeste = churnrf.randomSplit([0.8, 0.2])
print(churnTreino.count())
print(churnTeste.count())

8032


1968


In [7]:
from pyspark.ml.classification import LogisticRegression

logistic = LogisticRegression(featuresCol="independente", labelCol="dependente", maxIter=100, regParam=0.08)
modelo = logistic.fit(churnTreino)

In [10]:
resumo = modelo.summary

acuracia = resumo.accuracy
precisao = resumo.weightedPrecision
recall = resumo.weightedRecall
auc = resumo.areaUnderROC

print("Acurácio: ", acuracia, "\nPrecisão: ", precisao, "\nRecall: ", recall, "\nAUC: ", auc)

Acurácio:  0.8019173306772909 
Precisão:  0.7825838033043065 
Recall:  0.8019173306772909 
AUC:  0.7675743501159642


In [11]:
previsao = modelo.transform(churnTeste)
previsao.select("dependente", "prediction", "probability", "rawPrediction").show(5, truncate=False)

+----------+----------+----------------------------------------+----------------------------------------+
|dependente|prediction|probability                             |rawPrediction                           |
+----------+----------+----------------------------------------+----------------------------------------+
|1.0       |0.0       |[0.6888155792389475,0.31118442076105246]|[0.7945878415711711,-0.7945878415711711]|
|1.0       |0.0       |[0.7233012640058477,0.2766987359941523] |[0.960896502770773,-0.960896502770773]  |
|1.0       |0.0       |[0.7230621194656994,0.2769378805343006] |[0.9597019144687917,-0.9597019144687917]|
|1.0       |0.0       |[0.6836789171124569,0.3163210828875431] |[0.7707306051754932,-0.7707306051754932]|
|1.0       |0.0       |[0.675444165874503,0.32455583412549704] |[0.732912914624569,-0.732912914624569]  |
+----------+----------+----------------------------------------+----------------------------------------+
only showing top 5 rows



In [12]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

avaliar = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="dependente", metricName="areaUnderROC")
areaUnderRoc = avaliar.evaluate(previsao)
print(areaUnderRoc)

0.7615336129157005
